Code to run inferential tests on our data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import matplotlib.transforms as transforms
import matplotlib
import pandas as pd
import os
import os.path
from scipy import stats
import rpy2
import pingouin as pg

In [2]:
dname = os.getcwd()
dname

'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts\\part_level_analysis'

In [3]:
df = pd.read_csv(dname + "/params_trialwise/participantwise_log_likelihood_2.csv")
df.rename(columns={ df.columns[0]: "ID" }, inplace = True)
df.head(6)

,ID,sigmoid,linear,relu,linear_step_2
0,0,3.782378,1.512084,1.512084,9.458630
1,1,2.810269,1.860386,1.860386,13.867986
2,2,9.675852,1.405977,1.405977,9.424753
3,3,6.361855,1.948662,1.948662,10.460960
4,4,1.706993,1.183748,1.183748,6.267608
5,5,5.508092,1.057950,1.057950,11.016581


lets test for normality

In [4]:
for column in df:
    print(column)
    k2, p = stats.shapiro(df[column])
    print(k2, p)

ID
0.9556298851966858 0.06263722479343414
sigmoid
0.9523840546607971 0.0461496002972126
linear
0.9624005556106567 0.11893825232982635
relu
0.9614629149436951 0.10882913321256638
linear_step_2
0.934101402759552 0.008761046454310417


This looks okayish so lets run an ANOVA

In [5]:
# Call R
from rpy2.robjects.packages import importr
from rpy2 import robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()  # make pyr2 accept and auto-convert pandas dataframes
from rpy2.robjects import IntVector, FloatVector, FactorVector, globalenv
lme4 = importr('lme4')
base = importr('base')
stats = importr('stats')
broom = importr('broom')
utils = importr('utils')
ez = importr('ez')

Unable to determine R home: [WinError 2] The system cannot find the file specified
Unable to determine R library path: Command '('C:\\Program Files\\R\\R-4.1.2\\bin\\Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 1.


In [7]:
# df_long = pd.wide_to_long(df, stubnames = ["log-likelihood"], i = ["sigmoid", "linear", "relu", "linear_step"], j = "type")
df_long = pd.melt(df, id_vars = "ID", value_vars=["sigmoid", "linear", "relu", "linear_step_2"], value_name="log_likelihood", var_name="func_type")
# df_long.insert(0, 'ID', range(0, len(df_long)))
print(df_long.shape)
df_long.head(10)

(196, 3)


,ID,func_type,log_likelihood
0,0,sigmoid,3.782378
1,1,sigmoid,2.810269
2,2,sigmoid,9.675852
3,3,sigmoid,6.361855
4,4,sigmoid,1.706993
5,5,sigmoid,5.508092
6,6,sigmoid,12.693263
7,7,sigmoid,2.316006
8,8,sigmoid,7.226887
9,9,sigmoid,3.704647


In [9]:
globalenv['log_likelihood'] = FloatVector(df_long['log_likelihood'])
globalenv['func_type'] = FactorVector(df_long['func_type'])

In [10]:
anova1 = robjects.r("""
library(ez)
function(df) {
    # df gets passed in
    ezANOVA(
        data=df,
        dv=.(log_likelihood),
        within = .(func_type),
        wid=ID, 
        detailed=TRUE)
}
""")
print(anova1(df_long))

R[write to console]: Warning:
R[write to console]:  Converting "ID" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "func_type" to factor for ANOVA.



$ANOVA
       Effect DFn DFd      SSn      SSd        F            p p<.05       ges
1 (Intercept)   1  48 6091.455 1110.227 263.3605 4.030035e-21     * 0.7352425
2   func_type   3 144 3447.843 1083.279 152.7736 1.511690e-44     * 0.6111735

$`Mauchly's Test for Sphericity`
     Effect            W            p p<.05
2 func_type 1.110352e-05 1.64156e-112     *

$`Sphericity Corrections`
     Effect       GGe        p[GG] p[GG]<.05       HFe        p[HF] p[HF]<.05
2 func_type 0.6041406 6.271359e-28         * 0.6264936 7.207687e-29         *




ANOVA and post hoc test in with different syntax

In [12]:
res = pg.rm_anova(data = df_long, dv="log_likelihood", within="func_type", subject="ID", detailed=True)
res

,Source,SS,DF,MS,F,p-unc,p-GG-corr,ng2,eps,sphericity,W-spher,p-spher
0,func_type,3447.842627,3,1149.280876,152.773639,1.511690e-44,6.271359e-28,0.611174,0.604141,False,0.268136,6.102259e-12
1,Error,1083.278810,144,7.522770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Make post hocs

In [68]:
post_hocs = pg.pairwise_ttests(data=df_long, 
                              dv="log_likelihood", 
                              within="func_type", 
                              subject="ID", 
                              padjust="bonf")

post_hocs


c:\Users\Jirko\anaconda3\envs\simple_analysis\lib\site-packages\pingouin\pairwise.py:27: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,func_type,linear,linear_step,True,True,-43.672896,48.0,two-sided,2.659429e-40,1.595657e-39,bonf,5.813e+36,-7.024180
1,func_type,linear,relu,True,True,1.416619,48.0,two-sided,1.630520e-01,9.783120e-01,bonf,0.396,0.003027
2,func_type,linear,sigmoid,True,True,-9.635225,48.0,two-sided,8.427834e-13,5.056700e-12,bonf,9.828e+09,-1.740385
3,func_type,linear_step,relu,True,True,43.721401,48.0,two-sided,2.524669e-40,1.514802e-39,bonf,6.117e+36,7.024764
4,func_type,linear_step,sigmoid,True,True,20.545881,48.0,two-sided,1.985855e-25,1.191513e-24,bonf,1.722e+22,2.697608
5,func_type,relu,sigmoid,True,True,-9.638575,48.0,two-sided,8.334713e-13,5.000828e-12,bonf,9.934e+09,-1.740956
